In [11]:
import os
import sys
sys.path.append('/home/sensei/jupy-notebooks/Guestbook')
print("============================================")
print("/  Guestbook Suspensions is running.   /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import time

/  Guestbook Suspensions is running.   /


In [12]:
def getDbConnection(dbhost, dbport, dbname, dbuser, dbpass):
  conn_str = "host={0} port={1} dbname={2} user={3} password={4}".format(dbhost, dbport, dbname, dbuser, dbpass)
  try:
    conn = pg.connect(conn_str)
    return conn
  except pg.OperationalError:
    return None

In [13]:
print(os.getcwd())
config = configparser.ConfigParser()
config.read("../guestbook_secrets.ini")

_DB_HOST  = config['guestbookdb']['dbhost']
_DB_PORT  = config['guestbookdb']['dbport']
_DB_NAME  = config['guestbookdb']['dbname']
_DB_USER  = config['guestbookdb']['dbuser']
_DB_PASS  = config['guestbookdb']['dbpass']
    
_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])  
    
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

conn = getDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Guestbook database!")
else:
  print("You are not connected to the database.")

/home/gbadmin/jupy-notebooks/Reports
localhost 5432 guestbookdb postgres w0lfpack
Welcome to Jupyter Notebook.  You are connected to the Guestbook database!


In [14]:
# reset iscurrentlysuspended on all records
resetsql = "UPDATE guestbook_person \
            SET iscurrentlySuspended=false"

logger(_LOG_DEBUG, resetsql)
cur = conn.cursor()
cur.execute(resetsql)
# get the number of updated rows
updated_rows = cur.rowcount
logger(_LOG_DEBUG, "{} rows are reset".format(updated_rows))
# Commit the changes to the database
conn.commit()

UPDATE guestbook_person             SET iscurrentlySuspended=false
1216 rows are reset


In [15]:
currenttime = datetime.now()

currenttime = currenttime

suspensionsquery = "SELECT * \
             FROM guestbook_suspension \
             WHERE endtime>'{}'".format(currenttime)

logger(_LOG_DEBUG, suspensionsquery)
df = pd.read_sql(suspensionsquery, conn)

logger(_LOG_DEBUG, "number of active suspensions {}".format(len(df.index)))
for ind in df.index:
    personid = df['person_id'][ind]
    reasonid = df['reason_id'][ind]
    starttime = df['starttime'][ind]
    endtime = df['endtime'][ind]
    logger(_LOG_DEBUG, "person {} is suspended until {}".format(personid, endtime))
    # mark the person record 'iscurrentlysuspended=True'
    updatesql = "UPDATE guestbook_person \
                   SET iscurrentlySuspended=True \
                   WHERE idperson={}".format(personid)
    logger(_LOG_DEBUG, updatesql)
    cur = conn.cursor()
    cur.execute(updatesql)
    # get the number of updated rows
    updated_rows = cur.rowcount
    logger(_LOG_DEBUG, "{} rows are updated".format(updated_rows))
    # Commit the changes to the database
    conn.commit()
    

SELECT *              FROM guestbook_suspension              WHERE endtime>'2021-07-21 13:19:56.190970'
number of active suspensions 2
person 17 is suspended until 2021-07-25 03:59:59.160332+00:00
UPDATE guestbook_person                    SET iscurrentlySuspended=True                    WHERE idperson=17
1 rows are updated
person 17 is suspended until 2021-07-25 03:59:59.689805+00:00
UPDATE guestbook_person                    SET iscurrentlySuspended=True                    WHERE idperson=17
1 rows are updated
